In [1]:
    !pip install razdel transformers datasets accelerate seqeval evaluate

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import razdel
from razdel import tokenize
import string
import re
from transformers import AutoTokenizer, BertForTokenClassification, pipeline
import numpy as np
from datasets import load_dataset
import transformers
from accelerate import Accelerator
import torch
from torch import nn

In [ ]:
seed = 1488
transformers.set_seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
path_to_train_raw_data = "/content/drive/MyDrive/data/train_supervised_dataset.csv"

In [ ]:
df = pd.read_csv(path_to_train_raw_data)
df.head()

,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


In [ ]:
df = df.fillna("")

In [ ]:
#тут можно еще сделать ru_to_eng(дада и такие странные примеры есть)
def eng_to_ru(word: str) -> str:
    if len(word) <= 1:
        return word
    a = ord('а')
    ru_alphabet = ''.join([chr(i) for i in range(a,a+33)])
    eng_alphabet = string.ascii_lowercase[:26]
    change = {
        "a": "а",
        "e": "е",
        "o": "о",
        "k": "к",
        "3": "з",
        "p": "р",
        "c": "с",
        "m": "м",
        "x": "х",
        "t": "т",
        "y": "у",
        "z": "з",
    }

    new_word = ""
    for i in range(len(word)):
        curr = word[i]
        if curr in eng_alphabet:
            if i - 1 < 0:
                prev_letter = "j"
            else:
                prev_letter = word[i - 1]

            if i + 1 >= len(word):
                next_letter = "j"
            else:
                next_letter = word[i + 1]

            if next_letter in ru_alphabet or prev_letter in ru_alphabet:
                curr = change.get(curr, curr)
        new_word += curr

    return new_word
# to add: seperate by string.punctuations (add space between words)
#(probably i can just use some tokenizer and then make string of tokenizer output)


In [ ]:
def only_ones(word: str) -> str:
    if word.isdigit() or re.match(r'^-?\d+(?:\.\d+)$', word):
        return "1"
    else:
        new_word = ""
        i = 0
        while i < len(word):
            curr = word[i]
            while i < len(word) and word[i].isdigit():
                curr = "1"
                i += 1
            new_word += curr
            if curr != "1":
                i += 1

    return new_word


In [ ]:
def remove_punct(word: str) -> str:
    word = re.sub(r'[^\w\s]+', ' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word.strip()


def preprocess_string(word: str) -> str:
    new_word = ""


    word = remove_punct(word)

    if word != "товара нет":
        new_word = eng_to_ru(only_ones(word).lower())

    return new_word

In [ ]:
for i in range(1):
    print()
    q = tokenize(df.name[i])
    for x in q:
        print(x)



Substring(0, 6, 'Petmax')
Substring(7, 13, 'Бантик')
Substring(14, 21, 'леопард')
Substring(22, 23, 'с')
Substring(24, 29, 'красн')
Substring(30, 35, 'розой')
Substring(36, 37, '2')
Substring(37, 39, 'шт')


In [ ]:
names = df.name.apply(preprocess_string)
names

0                  petmax бантик леопард с красн розой 1шт
1                                 1 бусы для елки шарики_1
2                               футболка piazza italia wr1
3               1 yi1 1x one заколка для волос для девочки
4                                             одежда вес 1
                               ...                        
24995                                  вода саирме с г 1мл
24996                             моя семя 1 1л и ассортим
24997             рулет бисквитн яшкино клубничный со слив
24998    1 почвогрунт цветочное счастье фаско декоратив...
24999           семечки жар 1г крутой окер с солью гринвич
Name: name, Length: 25000, dtype: object

In [ ]:
def tokenize_names(names: pd.Series) -> pd.Series:
    return [list(map(lambda x: x.text, tokenize(x))) for x in names]

names = tokenize_names(names)
#delete solo symbols
for i in range(len(names)):
    new_name = []
    for j in range(len(names[i])):
        if len(names[i][j]) > 1:
            new_name.append(names[i][j])
    names[i] = new_name
names[:2]

[['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['бусы', 'для', 'елки', 'шарики_1']]

In [ ]:
brands = df.brand.apply(preprocess_string)
goods = df.good.apply(preprocess_string)

In [ ]:
brands[:10], goods[:10]

(0           petmax
 1                 
 2    piazza italia
 3                 
 4                 
 5        русалочка
 6                 
 7       очаковский
 8             cosy
 9              ekf
 Name: brand, dtype: object,
 0        бантик
 1          бусы
 2      футболка
 3       заколка
 4        одежда
 5         губка
 6          обои
 7          квас
 8         маска
 9    наконечник
 Name: good, dtype: object)

In [ ]:
list(tokenize(brands[2]))

[Substring(0, 6, 'piazza'), Substring(7, 13, 'italia')]

In [ ]:
#tagging every tokken if it has label or nah
# нужно текенизировать brand and goods
label_names = ["O", "B-BRAND", "I-BRAND", "B-GOOD", "I-GOOD"] # "O" stand for out of context or smth like this
skipped = 0
def label_name(name: list, brand: str, good: str) -> list:
    """
    gives label to every word in 1 receipt
    """
    labels = ["O" for _ in range(len(name))]
    brand = [x.text for x in tokenize(brand)]
    good = [x.text for x in tokenize(good)]
    numeric_labels = [0 for _ in range(len(name))]
    global skipped

    if brand:
        try:
            idx = name.index(brand[0])
            labels[idx] = label_names[1]
            numeric_labels[idx] = 1

            for x in brand[1:]:
                idx = name.index(x)
                labels[idx] = label_names[2]
                numeric_labels[idx] = 2
        except ValueError:
            skipped += 1
            return None

    if good:
        try:
            idx = name.index(good[0])
            labels[idx] = label_names[3]
            numeric_labels[idx] = 3

            for x in good[1:]:
                idx = name.index(x)
                labels[idx] = label_names[4]
                numeric_labels[idx] = 4
        except ValueError:
            skipped += 1
            return None
    return name, labels, numeric_labels



In [ ]:
i = 0
names[i]

['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт']

In [ ]:
label_name(names[i], brands[i], goods[i])

(['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['B-BRAND', 'B-GOOD', 'O', 'O', 'O', 'O'],
 [1, 3, 0, 0, 0, 0])

In [ ]:
name_and_labels = []
skipped = 0
real_brands = []
real_goods = []
for n, b, g in zip(names, brands, goods):
    #print (n, b)
    curr = label_name(n, b, g)
    if curr is not None:
        real_brands.append(b)
        real_goods.append(g)
        name_and_labels.append(curr)

In [ ]:
print(f"{skipped} of {len(df)} examples were skipped")

3734 of 25000 examples were skipped


In [ ]:
name_and_labels[:3]

[(['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
  ['B-BRAND', 'B-GOOD', 'O', 'O', 'O', 'O'],
  [1, 3, 0, 0, 0, 0]),
 (['бусы', 'для', 'елки', 'шарики_1'],
  ['B-GOOD', 'O', 'O', 'O'],
  [3, 0, 0, 0]),
 (['футболка', 'piazza', 'italia', 'wr'],
  ['B-GOOD', 'B-BRAND', 'I-BRAND', 'O'],
  [3, 1, 2, 0])]

In [ ]:
from datasets import Dataset, Features, Sequence, Value, DatasetDict

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification
model_name = "DeepPavlov/rubert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
sep_token = tokenizer.sep_token
sep_token_id = tokenizer.sep_token_id

model = AutoModelForTokenClassification.from_pretrained(model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [ ]:
# from transformers import BertForTokenClassification, BertTokenizer, BertForMaskedLM
# from torch import nn

# model_name = "/content/drive/MyDrive/models/bertfortokenclassification_mask_fill/bert-base-uncased-finetuned-imdb/checkpoint-2000"
# # Load pre-trained BERT model for masked language modeling
# mask_model = BertForMaskedLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# sep_token = tokenizer.sep_token
# sep_token_id = tokenizer.sep_token_id

# # Add a new classification layer for NER
# num_labels = 5 # Number of entity labels in your dataset
# model = BertForTokenClassification.from_pretrained(
#     "bert-base-uncased",
#     num_labels=num_labels,
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True
# )

In [ ]:
# model.bert = mask_model.bert
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [ ]:
# # Replace the final layer of the pre-trained BERT model with the new classification layer
# model.classifier = nn.Linear(768, num_labels)
# i should've run this code if out_features != num_labels

In [ ]:
inputs = tokenizer(names[0], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'pet',
 '##max',
 'бан',
 '##тик',
 'леопард',
 'крас',
 '##н',
 'роз',
 '##ой',
 'шт',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 0, 1, 1, 2, 3, 3, 4, 4, 5, None]

In [ ]:
model.config.id2label

{0: 'O', 1: 'B-BRAND', 2: 'I-BRAND', 3: 'B-GOOD', 4: 'I-GOOD'}

In [ ]:
preprocessed_dataset = pd.DataFrame(
    {
        "tokens": [x[0] for x in name_and_labels],
        "labels": [x[1] for x in name_and_labels],
        "numeric_labels": [x[2] for x in name_and_labels],
    }
)

In [ ]:
preprocessed_dataset.head()

,tokens,labels,numeric_labels
0,"[petmax, бантик, леопард, красн, розой, шт]","[B-BRAND, B-GOOD, O, O, O, O]","[1, 3, 0, 0, 0, 0]"
1,"[бусы, для, елки, шарики_1]","[B-GOOD, O, O, O]","[3, 0, 0, 0]"
2,"[футболка, piazza, italia, wr]","[B-GOOD, B-BRAND, I-BRAND, O]","[3, 1, 2, 0]"
3,"[yi, one, заколка, для, волос, для, девочки]","[O, O, B-GOOD, O, O, O, O]","[0, 0, 3, 0, 0, 0, 0]"
4,"[одежда, вес]","[B-GOOD, O]","[3, 0]"


In [ ]:
n = len(preprocessed_dataset)
train = preprocessed_dataset[:int(n * 8)]
val = preprocessed_dataset[int(n * 0.8):]
dataset_dict = {"train": train, "validation": val}

train_dataset = Dataset.from_dict(dataset_dict["train"])
validation_dataset = Dataset.from_dict(dataset_dict["validation"])

In [ ]:
train_dataset["tokens"][:3]

[['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['бусы', 'для', 'елки', 'шарики_1'],
 ['футболка', 'piazza', 'italia', 'wr']]

In [ ]:
inputs = tokenizer(train_dataset[0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'pet',
 '##max',
 'бан',
 '##тик',
 'леопард',
 'крас',
 '##н',
 'роз',
 '##ой',
 'шт',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 0, 1, 1, 2, 3, 3, 4, 4, 5, None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            # here shoud've been changing B-.. to I-... but i've already done it
            new_labels.append(label)

    return new_labels

In [ ]:
labels = train_dataset[0]["numeric_labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 3, 0, 0, 0, 0]
[-100, 1, 2, 3, 4, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["numeric_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_train = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

Map:   0%|          | 0/21266 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenized_val = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

Map:   0%|          | 0/4254 [00:00<?, ? examples/s]

In [ ]:
tokenized_train[0]

{'labels': [-100, 1, 2, 3, 4, 0, 0, 0, 0, 0, 0, -100],
 'input_ids': [101,
  33296,
  90269,
  9117,
  27976,
  92445,
  8170,
  858,
  11663,
  1519,
  5661,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Padding

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_train[i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    1,    2,    3,    4,    0,    0,    0,    0,    0,    0, -100],
        [-100,    3,    4,    0,    0,    0,    0,    0,    0, -100, -100, -100]])

# Metrics

In [ ]:
class F1Score:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def update(self, pred, target):
        pred = frozenset(x for x in pred)
        target = frozenset(x for x in target)
        self.tp += len(pred & target)
        self.fp += len(pred - target)
        self.fn += len(target - pred)

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def get(self):
        if self.tp == 0:
            return 0.0
        precision = self.tp / (self.tp + self.fp)
        recall = self.tp / (self.tp + self.fn)
        return 2 / (1 / precision + 1 / recall)


import evaluate

metric = evaluate.load("seqeval")


# Training

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    model_name,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

In [ ]:
validation_dataset[:2]

{'tokens': [['лопатка', 'малая', 'бук', 'шт'],
  ['провод', 'пвс', 'гост', 'белый', 'пог']],
 'labels': [['B-GOOD', 'O', 'O', 'O'], ['B-GOOD', 'O', 'O', 'O', 'O']],
 'numeric_labels': [[3, 0, 0, 0], [3, 0, 0, 0, 0]]}

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_train,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)
eval_dataloader = DataLoader(
    tokenized_val, collate_fn=data_collator, batch_size=16
)

# Hyperparameters search

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    f1_brand = results["BRAND"]["f1"]
    f1_good = results["GOOD"]["f1"]
    weighted_f1 = 2 / 3 * f1_brand + 1 / 3 * f1_good
    return {
        "f1": weighted_f1,
        "f1_brand": f1_brand,
    }

In [ ]:
#!pip install optuna

In [ ]:
# from transformers import Trainer

# args = TrainingArguments(
#     model_name,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     num_train_epochs=6
# )

# def optuna_hp_space(trial):

#     return {

#         "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),

#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),

#     }

# def model_init(trial):
#     return AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/models/deep_pavlov_init", id2label=id2label,
#                                                             label2id=label2id, ignore_mismatched_sizes=True)


In [ ]:
# trainer = Trainer(
#     model=None,
#     args=args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_val,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     model_init=model_init
# )

# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="optuna",
#     hp_space=optuna_hp_space,
#     n_trials=5,
# )


# Train(yes)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1.2396744163601523e-05)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 6
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "/content/drive/MyDrive/models/deep_pavlov_init"

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from tqdm.auto import tqdm
import torch



progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:

        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)



    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)


    results = metric.compute()
    f1_brand = results["BRAND"]["f1"]
    f1_good = results["GOOD"]["f1"]
    weighted_f1 = 2 / 3 * f1_brand + 1 / 3 * f1_good
    print(f"epoch № {epoch + 1}")
    print(f"WEIGHTED_F1 = {weighted_f1}")
    print(f"f1 brand = {f1_brand}")
    print(f"f1 good = {f1_good}")
    print(
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )
    print()

    # Save
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)



  0%|          | 0/7980 [00:00<?, ?it/s]

epoch № 1
WEIGHTED_F1 = 0.8780628242455035
f1 brand = 0.836874324810947
f1 good = 0.960439823114617
{'precision': 0.9342958161461402, 'recall': 0.8891069676153092, 'f1': 0.9111414409884347, 'accuracy': 0.9496976124279859}

epoch № 2
WEIGHTED_F1 = 0.9278158793583887
f1 brand = 0.9039716576542979
f1 good = 0.9755043227665706
{'precision': 0.95550972304066, 'recall': 0.9395914819643634, 'f1': 0.9474837484478854, 'accuracy': 0.9723981770758692}

epoch № 3
WEIGHTED_F1 = 0.9549030985864296
f1 brand = 0.9413489736070381
f1 good = 0.982011348545213
{'precision': 0.9774602239245728, 'recall': 0.9545389152639908, 'f1': 0.9658635999708859, 'accuracy': 0.9806529278569176}

epoch № 4
WEIGHTED_F1 = 0.9736413431818274
f1 brand = 0.967935495968498
f1 good = 0.9850530376084861
{'precision': 0.9821744254566883, 'recall': 0.9745651220581786, 'f1': 0.9783549783549783, 'accuracy': 0.9894236005617817}

epoch № 5
WEIGHTED_F1 = 0.9797748442930514
f1 brand = 0.9751541191855035
f1 good = 0.9890162945081473
{'pr

In [ ]:
0.995406071392865

In [ ]:
for g in optimizer.param_groups:
    g['lr'] = 2e-6

NameError: ignored

# Evaluating

In [ ]:
test_dataset = list(map(lambda x: [' '.join(x[0])] + [x[i] for i in range(1, len(x))], name_and_labels[int(n * 0.75):]))
test_dataset[:3]

[['платье boa', ['B-GOOD', 'O'], [3, 0]],
 ['ролик пр cape csef', ['B-GOOD', 'O', 'O', 'O'], [3, 0, 0, 0]],
 ['sheba pleasure гр кошек ломт соусе курица',
  ['B-BRAND', 'O', 'O', 'O', 'O', 'O', 'B-GOOD'],
  [1, 0, 0, 0, 0, 0, 3]]]

In [ ]:
from transformers import pipeline
device = 0 if torch.cuda.is_available() else None
token_classifier = pipeline(
    "token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0
)

In [ ]:
out_brands = []
out_goods = []
for name, label, ner_tags in test_dataset:
    curr_b = []
    curr_g = []
    out = token_classifier(name)
    for x in out:
        if x["entity_group"] == "BRAND":
            curr_b.append(x["word"])
        else:
            curr_g.append(x["word"])
    out_brands.append(curr_b)
    out_goods.append(curr_g)

In [ ]:
out_goods = list(map(lambda x: ','.join(x), out_goods))
out_goods[:10]

['платье',
 'ролик',
 'курица',
 'нож',
 'шоколад',
 'краска',
 'ветчина',
 'шоколад',
 'продукт плавленый',
 'скребок']

In [ ]:
out_brands = list(map(lambda x: ','.join(x), out_brands))
out_brands[:10]

['',
 '',
 'sheba',
 'rexant',
 'milka',
 'kapous',
 'сибирские колбасы',
 'schogetten',
 'дружба',
 '']

In [ ]:
# len(out_brands)

In [ ]:
f1_brand = F1Score()
f1_good = F1Score()
f1_brand.update(out_brands, real_brands[int(n * 0.75):])
f1_good.update(out_goods, real_goods[int(n * 0.75):])

f1 = 2 / 3 * f1_brand.get() + 1 / 3 * f1_good.get()
f1, f1_brand.get(), f1_good.get()

(0.9893016300785078, 0.989735020291239, 0.9884348496530455)

оказывается метрика правильная. Похоже проблема в предобработке данных. Попробуем посчитать ф1 на сырых данных

# test

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/data/test_dataset.csv")
test_data.head()

,id,name
0,0,"469-210 ЕРМАК Клей универсальный, 15мл, блистер"
1,1,Торт СЛАДУШКА Зимняя вишня 700г
2,2,"Смеситель ""CALORIE"" 1023 А06 д/кухни"
3,3,Лимон 50гр БАР
4,4,"Коньяк САРАДЖИШВИЛИ 5 лет 0,5л Грузия"


In [ ]:
test_data.name = test_data.name.apply(preprocess_string)
test_data.head()

,id,name
0,0,1 1 ермак клей универсальный 1мл блистер
1,1,торт сладушка зимняя вишня 1г
2,2,смеситель calorie 1 а1 д кухни
3,3,лимон 1гр бар
4,4,коньяк сараджишвили 1 лет 1 1л грузия


In [ ]:
from transformers import pipeline
token_classifier = pipeline(
    "token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="average", device=0
)

In [ ]:
brands = []
goods = []
i = 0
for x in test_data.name:
    curr_b = []
    curr_g = []
    out = token_classifier(x)
    for x in out:
        if x["entity_group"] == "BRAND":
            curr_b.append(x["word"])
        else:
            curr_g.append(x["word"])
    brands.append(curr_b)
    goods.append(curr_g)

In [ ]:
subm = pd.DataFrame(
    {
        "id": [i for i in range(len(brands))],
        "good": goods,
        "brand": brands
    }
)
subm.head()

,id,good,brand
0,0,[клей],[ермак]
1,1,[торт],[сладушка]
2,2,[смеситель],[calorie]
3,3,[лимон],[]
4,4,[коньяк],[сараджишвили]


In [ ]:
subm.good = subm.good.apply(lambda x: ','.join(x))
subm.brand = subm.brand.apply(lambda x: ','.join(x))
subm.head()

,id,good,brand
0,0,клей,ермак
1,1,торт,сладушка
2,2,смеситель,calorie
3,3,лимон,
4,4,коньяк,сараджишвили


In [ ]:
subm.to_csv("deep_pavlov_average.csv")

In [ ]:
subm.brand[30:60]

30                             
31                             
32                     wrangler
33          четвероногий гурман
34                             
35                             
36                             
37                             
38                             
39                   yasiyilian
40                             
41                             
42                             
43                  жигулёвское
44                love republic
45                             
46                  schwarzkopf
47                    nakamichi
48                             
49                      бавария
50                             
51    confetti bella,margherita
52                             
53                             
54                             
55                             
56                         tina
57                      tai tau
58                самое любимое
59                             
Name: brand, dtype: object

In [ ]:
subm.brand[30:60]

30                             
31                             
32                     wrangler
33          четвероногий гурман
34                             
35                         вико
36                             
37                             
38                             
39                   yasiyilian
40                             
41                             
42                             
43                  жигулёвское
44                love republic
45                             
46                  schwarzkopf
47                    nakamichi
48                             
49                      бавария
50                             
51    confetti bella,margherita
52                             
53                       тархун
54                             
55                             
56                         tina
57                      tai tau
58                самое любимое
59                    альфасорб
Name: brand, dtype: object

In [ ]:
test_data.name[51]

'коврик для в к confetti bella из 1 шт 1х1см margherita бежевый 1 1'